In [ ]:
import numpy as np
import parcels as parcels
from parcels import StatusCode
from datetime import datetime, timedelta
import xarray as xr
import math
import matplotlib.pyplot as plt

In [ ]:
xmin, xmax, dx = 0, 100, 1
ymin, ymax, dy = 0, 200, 1
tmin, tmax, dt = 0, 100, 1

x = np.arange(xmin, xmax + 0.5*dx, dx)
y = np.arange(ymin, ymax + 0.5*dy, dy)
# times = np.arange(tmin, tmax, dt)

X, Y = np.meshgrid(x, y)

# Define U: 1 if x < 80, else NaN
U = np.where(X < 80, 1.0, np.nan)
V = 0 * Y
beach = np.where(X < 80, 0,1)

dset = xr.Dataset(
                
data_vars=dict(U=(['lat','lon'],U),
            V=(['lat','lon'],V)),
            coords=dict(
                    lon=('lon',x),
                    lat=('lat',y),
                   

            )
)

dset_beach = xr.Dataset(
                
data_vars=dict(beach=(['lat','lon'],beach)),
            coords=dict(
                    lon=('lon',x),
                    lat=('lat',y),
                   

            )
)
            

In [ ]:
fieldset =parcels.FieldSet.from_xarray_dataset(dset, variables = {'U':'U','V':'V'},dimensions={'lon':'lon','lat':'lat'},allow_time_extrapolation=True,mesh='flat')
beach = parcels.Field.from_xarray(dset_beach.beach,name='beach',dimensions={'lon':'lon','lat':'lat'},mesh='flat',allow_time_extrapolation=True)
fieldset.add_field(beach)

In [ ]:


def check_beach(particle,fieldset,time):
    b = fieldset.beach[time, particle.depth, particle.lat, particle.lon]
    
    if b > 0.8 : 
        particle.beached=1
        # particle.state= StatusCode.StopExecution
        # particle.delete()

def stop_moving(particle,fieldset,time):
    if(particle.beached ==1):
        particle_dlon = 0
        particle_dlat = 0 
        


PlasticParticle = parcels.JITParticle.add_variable([
    parcels.Variable('beached', dtype=np.int32, initial=0, to_write=True),
    parcels.Variable('wind_coefficient', dtype=np.float32, initial=0.03),      # default 3%
])

lons_particles = [70, 77.1, 80, 80.5, 90]
lats_particles = [25, 50, 75, 100, 125]

pset = parcels.ParticleSet(
    fieldset=fieldset,
    pclass=PlasticParticle,
    lon=lons_particles,
    lat=lats_particles
)

pfile = pset.ParticleFile(
    name='test_arno_output.zarr', outputdt=1)


# AdvectionRK4 is most standard advection in 2d (lon,lat)
kernels = [parcels.AdvectionRK4,check_beach, stop_moving]

########### RUN SIMULATIONS #############################
# use execute to run the simulation
pset.execute(kernels, runtime=100, dt=0.1, output_file=pfile)

In [ ]:
ds = xr.open_dataset('test_arno_output.zarr',engine='zarr')

In [ ]:
fig, ax = plt.subplots()
ax.plot(ds.lon.T,ds.lat.T,'--o')
# ax.set_xlim(0,100)
# ax.set_ylim(0,200)

In [ ]:
ds.beached.values

In [ ]:
ds.time

In [ ]:
ds.lon.values

In [ ]:
fig, ax = plt.subplots()
ax.plot(ds.lon.T,ds.beached.T,'o')